<h1>importing necessary libraries</h1>

In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

<h3>importing dataset</h3>

In [21]:
df = pd.read_csv("Pune_House_Data.csv")
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   area_type      13320 non-null  object 
 1   availability   13320 non-null  object 
 2   size           13304 non-null  object 
 3   society        7818 non-null   object 
 4   site_location  13319 non-null  object 
 5   total_sqft     13320 non-null  object 
 6   bath           13247 non-null  float64
 7   balcony        12711 non-null  float64
 8   price          13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


area_type           0
availability        0
size               16
society          5502
site_location       1
total_sqft          0
bath               73
balcony           609
price               0
dtype: int64

<h3>Keeping the necessary values and droppping the rest values</h3>

In [22]:
df = df.drop(columns=["area_type", "availability", "society"])

<h3>finding the missing values in the dataset and fill them.</h3>

In [23]:
df[["size", "site_location"]] = df[["size", "site_location"]].fillna(method="ffill")
df["bath"] = df["bath"].replace(np.nan, df["bath"].mean())
df["balcony"] = df["balcony"].replace(np.nan, df["balcony"].mean())
df

,size,site_location,total_sqft,bath,balcony,price
0,2 BHK,Alandi Road,1056,2.0,1.000000,39.07
1,4 Bedroom,Ambegaon Budruk,2600,5.0,3.000000,120.00
2,3 BHK,Anandnagar,1440,2.0,3.000000,62.00
3,3 BHK,Aundh,1521,3.0,1.000000,95.00
4,2 BHK,Aundh Road,1200,2.0,1.000000,51.00
...,...,...,...,...,...,...
13315,5 Bedroom,Pashan,3453,4.0,0.000000,231.00
13316,4 BHK,Paud Road,3600,5.0,1.584376,400.00
13317,2 BHK,Pirangut,1141,2.0,1.000000,60.00
13318,4 BHK,Prabhat Road,4689,4.0,1.000000,488.00


<h3>convertng the string data into numeric format to train the model</h3>

In [24]:
# areaString = df["area_type"].unique()
sizeString = df["size"].unique()
locationString = df["site_location"].unique()
sqFeetString = df["total_sqft"].unique()

enc = LabelEncoder()

for col in df.columns[:-3]: 
    df[col] = df[col].astype(str)
    df[col] = enc.fit_transform(df[col])

# areaNumeric = df["area_type"].unique()
sizeNumeric = df["size"].unique()
locationNumeric = df["site_location"].unique()
sqFeetNumeric = df["total_sqft"].unique()



<h3>creating dictionaries to store the values of numeric values and string values respectively</h3>

In [25]:
sizeDict, locationDict = {}, {}

# for key,value in zip(areaNumeric, areaString):
#     areaDict[key] = value

for key,value in zip(sizeNumeric, sizeString):
    sizeDict[key] = value

for key,value in zip(locationNumeric, locationString):
    locationDict[key] = value



<h3>Dividing the dataset into three parts that is train, validation and test .</h3>

In [26]:
np.random.seed(42)
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])
test

,size,site_location,total_sqft,bath,balcony,price
10773,16,20,1060,4.0,2.0,151.0
13297,13,48,74,2.0,1.0,52.0
147,13,51,2014,2.0,1.0,35.0
1668,16,36,90,2.0,1.0,45.0
11776,13,63,2006,2.0,1.0,49.0
...,...,...,...,...,...,...
11964,22,59,239,4.0,1.0,99.0
5191,16,6,642,3.0,2.0,120.0
5390,13,13,74,2.0,1.0,48.5
860,24,92,239,4.0,2.0,125.0


<h3>Scailing the dataset</h3>

In [27]:
def scale_data(dataFrame):
    x = dataFrame[dataFrame.columns[:-1]].values
    y = dataFrame[dataFrame.columns[-1]].values
    # print(type(y))
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
    data = np.hstack((x, np.reshape(y, (-1, 1))))
    return data, x, y
# x = df[df.columns[:-1]].values
# y = df[df.columns[-1]].values
# data = np.hstack((x, np.reshape(y, (-1, 1))))

In [28]:
train, X_train, Y_train = scale_data(train)
validate, X_validate, Y_validate = scale_data(valid)
test, X_test, Y_test = scale_data(test)

# print(test)